<a href="https://colab.research.google.com/github/nashwaisya/ams/blob/main/Latent_Dirichlet_Allocation_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalasi Modul

In [1]:
!pip install nltk
!pip install Sastrawi
!pip install regex
!pip install unidecode
!pip install textblob
!pip install tqdm
!pip install scapy
!pip install python-crfsuite
!pip install gensim
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 10.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scapy

## Import Modul

In [2]:
import pandas as pd
import numpy as np
import string
import nltk

# modul stemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stemmer = StemmerFactory().create_stemmer()

# modul preprocessing
nltk.download('punkt')
import re
from unidecode import unidecode
from html import unescape
from textblob import TextBlob
from tqdm import tqdm

# modul lemma & pos-tag
import spacy
from spacy.lang.id import Indonesian
from nltk.tag import CRFTagger
ct = CRFTagger()

# modul LDA
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

# modul DRT
from sklearn.manifold import TSNE

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Load Data

In [99]:
df = pd.read_csv('https://raw.githubusercontent.com/nashwaisya/ams/main/df_1%20(2).csv')
df.head()

,title,href,body,date
0,"Sahabat Ganjar Jambi on Instagram: ""Ganjar pem...",https://www.instagram.com/p/Cm3KrJSPKey/,"0 Likes, 0 Comments - Sahabat Ganjar Jambi (@s...",2023-01-01
1,pinterpolitik on Instagram: Dengan rendahnya e...,https://www.instagram.com/p/Cm8rQNngOkW/,"Dengan rendahnya elektabilitas Puan Maharani, ...",2023-01-02
2,asumsico on Instagram: Ganjar Pranowo disebut ...,https://www.instagram.com/p/Cm8f8KtPPnH/,asumsico • Original audio. Original audio. 692...,2023-01-02
3,"Asumsi on Instagram: ""Indikator Politik Indone...",https://www.instagram.com/p/Cm_i1VqvQLH/,Indikator Politik Indonesia merilis survei ele...,2023-01-03
4,pinterpolitik on Instagram: Dengan rendahnya e...,https://www.instagram.com/p/Cm8rQNngOkW/,"Dengan rendahnya elektabilitas Puan Maharani, ...",2023-01-03


## Preprocessing Data

### Load Slang atau Singkatan
Modifikasi sesuka hati jika sekiranya masih terdapat singkatan yang perlu diubah.

In [33]:
import urllib.request

# membaca data slang dari URL Github
url = 'https://raw.githubusercontent.com/taudataid/eLearning/master/data/slang.dic'
response = urllib.request.urlopen(url)
slang_id = eval(response.read())

# mencetak dictionary dalam format yang diinginkan
print("{")
for key, value in slang_id.items():
    print(f"    \"{key}\": \"{value}\",")
print("}")

{
    "1pun": "satupun",
    "7an": "tujuan",
    "Dr.": "doktor",
    "dr.": "dokter",
    "drg.": "dokter gigi",
    "Jkt": "Jakarta",
    "Jkw": "jokowi",
    "Napza": "narkoba psikotropika dan zat adiktif",
    "Nasihat": "nasehat",
    "ababil": "abg labil",
    "abis": "habis",
    "acc": "accord",
    "accord": "sesuai",
    "ad": "ada",
    "adl": "adalah",
    "adlah": "adalah",
    "adlh": "adalah",
    "administ": "administrasi",
    "adoh": "aduh",
    "afaik": "as far as i know",
    "agma": "agama",
    "aha": "tertawa",
    "ahaha": "haha",
    "ahiok": "ahok",
    "mehong": "mahal",
    "ahoax": "ahok",
    "ahokncc": "ahok",
    "aing": "saya",
    "aj": "saja",
    "aja": "saja",
    "ajak2": "ajak-ajak",
    "ajep-ajep": "dunia gemerlap",
    "ajj": "saja",
    "ak": "aku",
    "aka": "dikenal juga sebagai",
    "akherat": "akhirat",
    "akhirx": "akhirnya",
    "akika": "aku",
    "akko": "aku",
    "akkoh": "aku",
    "akku": "aku",
    "akn": "akan",
    "aktifis

### Load Stopword
Modifikasi sesuka hati jika sekiranya masih terdapat kata yang seharusnya dihilangkan.

In [34]:
import requests
import re

url = "https://raw.githubusercontent.com/taudataid/eLearning/master/data/stopwords_id.txt"
response = requests.get(url)
stopwords = set(re.findall(r'\w+', response.text))

# Tambahkan set tambahan ke dalam set stopwords
tambahan = set(['a', 'akan', 'atas', 
                'b', 'bisa', 'bahwa', 'bacapres', 'banyak',
                'c', 'comment', 'comments',
                'd', 'dari', 'di', 'diri',
                'e', 
                'f', 'followers', 'following', 'front', 
                'g', 
                'h',
                'i', 'instagram', 'inilahcom',
                'j', 'jateng', 'jalan', 
                'k',
                'l', 'like', 'likes', 'lalu',
                'm', 'menjadi', 'mulai', 'makin', 'meski',
                'n', 
                'o', 
                'p', 'photos', 'posts', 'punya',
                'q', 
                'r', 'rizieq',
                's', 'sebagai', 'saja', 'sama', 'orang', 'salah', 'selalu', 'satu', 'sindonews', 'saling', 'see',
                't', 'tribun',
                'u', 
                'v', 
                'w', 
                'x', 
                'y',
                'z',])
stopwords = stopwords.union(tambahan)


# Meletakkan setiap kata dalam kurung kurawal dan memberikan tanda kutip
stopwords_id = "{" + ", ".join([f'"{word}"' for word in stopwords]) + "}"

print(len(stopwords))

791


### NLP (Natural Language Preprocessing)
Membersihkan data teks dari karakter-karakter yang tidak diperlukan serta menangani stopword dan slang atau singkatan yang sudah di load sebelumnya dan lain sebagainya.

Note : setiap platform media sosial memiliki cara pembersihan data nya masing-masing (modifikasi sesuka hati).

In [100]:
def cleanbody(text):
    
    # menghapus url
    url_pattern = re.compile(r'(\w+:\/\/\S+)')
    text = url_pattern.sub(' ', text)
    
    # menghapus hashtag
    hashtag_pattern = re.compile(r'#\w+\b')
    text = hashtag_pattern.sub(' ', text)
    
    # menghapus nama pengguna instagram (memuat "@")
    username_pattern = re.compile(r'@\w+\b\s*')
    text = username_pattern.sub(' ', text)
    
    # menghapus angka
    text = re.sub(r'\b\d+\b|[^\w\s]', '', text)
    
    # mengahpus simbol
    symbol_pattern = re.compile(r'[^\w\s]+')
    text = symbol_pattern.sub(' ', text)
    
    # menghapus karakter yang tidak diperlukan (tergantung media sosial)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\r', ' ', text)
    text = re.sub(r'&\w+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # menangani huruf kapital dan spasi
    text = unidecode(unescape(text.lower().strip()))
    
    # menangani slang atau singkatan 
    Tokens = TextBlob(text).words
    for i,t in enumerate(Tokens):
      if t in slang_id.keys():
        Tokens[i] = slang_id[t]
        
    # menangani stopword
    text = ' '.join([t for t in Tokens if str(t) not in stopwords_id and len(t)>2])
    
    # stemming
    text = stemmer.stem(text)
    
    
    return text

In [101]:
# aplikasikan fungsi ke dalam kolom variabel baru
df['clean_body'] = ''
for idx, post in tqdm(df.iterrows()):
    df.at[idx, 'clean_body'] = cleanbody(post.body)

40it [00:11,  3.60it/s]


In [102]:
df.head()

,title,href,body,date,clean_body
0,"Sahabat Ganjar Jambi on Instagram: ""Ganjar pem...",https://www.instagram.com/p/Cm3KrJSPKey/,"0 Likes, 0 Comments - Sahabat Ganjar Jambi (@s...",2023-01-01,sahabat ganjar jambi ganjar pimpin sosok rakyat
1,pinterpolitik on Instagram: Dengan rendahnya e...,https://www.instagram.com/p/Cm8rQNngOkW/,"Dengan rendahnya elektabilitas Puan Maharani, ...",2023-01-02,rendah elektabilitas puan maharani nilai parta...
2,asumsico on Instagram: Ganjar Pranowo disebut ...,https://www.instagram.com/p/Cm8f8KtPPnH/,asumsico • Original audio. Original audio. 692...,2023-01-02,asumsico original audio original audio
3,"Asumsi on Instagram: ""Indikator Politik Indone...",https://www.instagram.com/p/Cm_i1VqvQLH/,Indikator Politik Indonesia merilis survei ele...,2023-01-03,indikator politik indonesia rilis survei elekt...
4,pinterpolitik on Instagram: Dengan rendahnya e...,https://www.instagram.com/p/Cm8rQNngOkW/,"Dengan rendahnya elektabilitas Puan Maharani, ...",2023-01-03,rendah elektabilitas puan maharani nilai parta...


### Lemmatisasi & Pos-Tag
Bagian preprocessing yang paling vital atau krusial untuk proses LDA dari yang lainnya, yaitu identifikasi kata dasar serta pengelompokkan kata berdasarkan kategori kata tersebut.

In [103]:
# lemma & pos-tag bahasa indonesia
nlp_id = Indonesian()
!wget -P data/ https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/all_indo_man_tag_corpus_model.crf.tagger
ct.set_model_file('data/all_indo_man_tag_corpus_model.crf.tagger')

--2023-05-08 15:41:27--  https://raw.githubusercontent.com/taudata-indonesia/eLearning/master/data/all_indo_man_tag_corpus_model.crf.tagger
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1722780 (1.6M) [application/octet-stream]
Saving to: ‘data/all_indo_man_tag_corpus_model.crf.tagger.7’

all_indo_man_tag_co 100%[===================>]   1.64M  --.-KB/s    in 0.01s   

2023-05-08 15:41:27 (132 MB/s) - ‘data/all_indo_man_tag_corpus_model.crf.tagger.7’ saved [1722780/1722780]



In [104]:
def NLPfilter(t, filters):

  # lemmatisasi
  tokens = nlp_id(t)

  # tokenisasi
  tokens = [str(k) for k in tokens if len(k)>2]

  # pos-tag
  hasil = ct.tag_sents([tokens])
  
  return [k[0] for k in hasil[0] if k[1] in filters]

  Note :
- NN : kata benda tunggal (meja, buku, kucing, cinta, ...)
- NNP : kata benda tunggal khusus (indonesia, google, nike, tokyo, ...)
- NNS : kata benda jamak (buku-buku, meja-meja, ...)
- NNPS : kata benda jamak khusus (beatles, avengers, simpsons, ...)
- JJ : kata sifat (marah, tinggi, besar, indah, ...)

In [105]:
# ambil variabel kolom hasil preprocessing
data = df['clean_body'].values

# pilih kategori kata 
filters = set(['NN', 'NNP', 'NNS', 'NNPS', 'JJ'])

# aplikasikan fungsi ke dalam dataframe baru
data_postTag = []
for i, d in tqdm(enumerate(data)):
    data_postTag.append(NLPfilter(d,filters))

' '.join(data_postTag[0])

40it [00:00, 1727.63it/s]


'sahabat jambi pimpin sosok rakyat'

In [106]:
# tokenisasi data kembali
data = [d for d in data_postTag if d]

## LDA (Latent Dirichlet Allocation)


In [107]:
# membuat representasi dictionary dari dokumen

# membuang token yang langka dan umum
dictionary_t = Dictionary(data)
dictionary_t.filter_extremes(no_below=2, no_above=0.90)

# membuat dictionary dan corpus yang diperlukan topic modelling
corpus_t = [dictionary_t.doc2bow(doc) for doc in data]
corpus_t = [t for t in corpus_t if t] # membuang corpus atau dokumen yang kosong

print('Number of unique tokens: %d' % len(dictionary_t))
print('Number of documents: %d' % len(corpus_t))
print(corpus_t[:1])

Number of unique tokens: 145
Number of documents: 40
[[(0, 1), (1, 1)]]


Pembuatan dataframe hasil keluaran dari algoritma LDA (Latent Dhiriclet Allocation).

In [108]:
def format_topics_sentences(ldamodel, corpus, texts, dates):
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = pd.concat([
                    sent_topics_df,                   
                pd.DataFrame([[int(topic_num), round(prop_topic, 4), topic_keywords]],
                             columns=["Dominant_Topic", "Perc_Contribution", "Topic_Keywords"])],
                    ignore_index=True,
                )
            else:
                break
    sent_topics_df.columns = ["Dominant_Topic", "Perc_Contribution", "Topic_Keywords"]

    # Add original text to the end of the output
    contents = pd.Series(texts)

    sent_topics_df = pd.concat([sent_topics_df, contents, pd.Series(dates)], axis=1)
    return sent_topics_df

Model DRT (Dimensional Reduction Technique).

In [109]:
def tsne_analysis(ldamodel, corpus):
    topic_weights = []
    for i, row_list in enumerate(ldamodel[corpus]):
        topic_weights.append([w for i, w in row_list])

    # Array of topic weights
    df_topics = pd.DataFrame(topic_weights).fillna(0).values

    # Keep the well separated points (optional)
    # arr = arr[np.amax(arr, axis=1) > 0.35]

    # Dominant topic number in each doc
    topic_nums = np.argmax(df_topics, axis=1)

    # tSNE Dimension Reduction
    try:
        tsne_model = TSNE(
            n_components=2, verbose=1, random_state=0, angle=0.99, init="pca"
        )
        tsne_lda = tsne_model.fit_transform(df_topics)
    except:
        print("TSNE_ANALYSIS WENT WRONG, PLEASE RE-CHECK YOUR BANK DATASET")
        return (topic_nums, None)

    return (topic_nums, tsne_lda)

Penggabungan dataframe sedemikian sehingga data ini lah yang nantinya di aplikasikan ke dalah dashboard LDA.

In [112]:
def lda_analysis(df):
    
    docs = list(df["clean_body"].values)

    punctuations = string.punctuation

    processed_docs = data
    print("Jumlah corpus atau dokumen", len(processed_docs))
    if len(processed_docs) < 11:
        print("INSUFFICIENT DOCS TO RUN LINEAR DISCRIMINANT ANALYSIS")
        return (None, None, None, None)

    print("Jumlah BoW (Bag of Words) corpus", len(corpus_t))
    print("Jumlah dictionary", len(list(dictionary_t.keys())))
    if len(list(dictionary_t.keys())) < 1:
        print("INSUFFICIENT DICTS TO RUN LINEAR DISCRIMINANT ANALYSIS")
        return (None, None, None, None)

    lda_model = LdaModel(
        corpus_t, num_topics=5, id2word=dictionary_t, passes=10
    )

    df_topic_sents_keywords = format_topics_sentences(
        ldamodel=lda_model,
        corpus=corpus_t,
        texts=docs,
        dates=list(df["date"].values),
    )
    print("Jumlah data", len(df_topic_sents_keywords))
    print("Data", df_topic_sents_keywords.head())
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = [
        "Document_No",
        "Dominant_Topic",
        "Topic_Perc_Contrib",
        "Keywords",
        "Text",
        "Date",
    ]

    print("Hasil DRT")
    topic_num, tsne_lda = tsne_analysis(lda_model, corpus_t)

    return (tsne_lda, lda_model, topic_num, df_dominant_topic)

In [113]:
# aplikasikan fungsi dan simpan hasilnya ke dalam beberapa variabel berikut
tsne_lda, lda_model, topic_num, df_dominant_topic = lda_analysis(df)

Jumlah corpus atau dokumen 40
Jumlah BoW (Bag of Words) corpus 40
Jumlah dictionary 145
Jumlah data 40
Data    Dominant_Topic  Perc_Contribution  \
0               2             0.7317   
1               3             0.9495   
2               1             0.8667   
3               0             0.9692   
4               3             0.9495   

                                      Topic_Keywords  \
0  partai, juang, indonesia, demokrasi, presiden,...   
1  partai, ketua, pilpres, demokrasi, indonesia, ...   
2  audio, original, acara, pranowo, festival, kha...   
3  elektabilitas, survei, pranowo, hasil, anies, ...   
4  partai, ketua, pilpres, demokrasi, indonesia, ...   

                                                   0           1  
0    sahabat ganjar jambi ganjar pimpin sosok rakyat  2023-01-01  
1  rendah elektabilitas puan maharani nilai parta...  2023-01-02  
2             asumsico original audio original audio  2023-01-02  
3  indikator politik indonesia rilis survei el

In [114]:
# data untuk pembuatan dashboard
df_dominant_topic = df_dominant_topic.dropna()
df_dominant_topic

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Date
0,0,2,0.7317,"partai, juang, indonesia, demokrasi, presiden,...",sahabat ganjar jambi ganjar pimpin sosok rakyat,2023-01-01
1,1,3,0.9495,"partai, ketua, pilpres, demokrasi, indonesia, ...",rendah elektabilitas puan maharani nilai parta...,2023-01-02
2,2,1,0.8667,"audio, original, acara, pranowo, festival, kha...",asumsico original audio original audio,2023-01-02
3,3,0,0.9692,"elektabilitas, survei, pranowo, hasil, anies, ...",indikator politik indonesia rilis survei elekt...,2023-01-03
4,4,3,0.9495,"partai, ketua, pilpres, demokrasi, indonesia, ...",rendah elektabilitas puan maharani nilai parta...,2023-01-03
5,5,1,0.8667,"audio, original, acara, pranowo, festival, kha...",asumsico original audio original audio,2023-01-03
6,6,1,0.8852,"audio, original, acara, pranowo, festival, kha...",fajarcoid jakarta amat politik rocky gerung is...,2023-01-04
7,7,0,0.9692,"elektabilitas, survei, pranowo, hasil, anies, ...",indikator politik indonesia rilis survei elekt...,2023-01-04
8,8,1,0.8852,"audio, original, acara, pranowo, festival, kha...",fajarcoid jakarta amat politik rocky gerung is...,2023-01-05
9,9,0,0.9578,"elektabilitas, survei, pranowo, hasil, anies, ...",hasil survei baru indikator politik indonesia ...,2023-01-05


In [115]:
topic_top3words = [
    (i, topic)
    for i, topics in lda_model.show_topics(formatted=False)
    for j, (topic, wt) in enumerate(topics)
    if j < 3
]

df_top3words_stacked = pd.DataFrame(
    topic_top3words, columns=['topic_id', 'words']
)
df_top3words = df_top3words_stacked.groupby("topic_id").agg(", ".join)
df_top3words.reset_index(level=0, inplace=True)

df_top3words

,topic_id,words
0,0,"elektabilitas, survei, pranowo"
1,1,"audio, original, acara"
2,2,"partai, juang, indonesia"
3,3,"partai, ketua, pilpres"
4,4,"partai, tim, indonesia"


In [116]:
tsne_df = pd.DataFrame({
    "tsne_x": tsne_lda[:, 0],
    "tsne_y": tsne_lda[:, 1],
    "topic_num": topic_num,
    "doc_num": df_dominant_topic["Document_No"],
}
)

In [117]:
tsne_df

,tsne_x,tsne_y,topic_num,doc_num
0,-9.933064,6.218514,2,0
1,-7.815633,7.840113,3,1
2,-8.037251,6.634380,1,2
3,-9.650216,8.066313,0,3
4,-7.758492,7.831347,3,4
5,-8.037251,6.634380,1,5
6,-7.941366,6.597940,1,6
7,-9.924706,7.831789,0,7
8,-7.945008,6.580053,1,8
9,-9.591493,7.810001,0,9


In [32]:
from google.colab import files

df_dominant_topic.to_csv('tsne_df.csv', index=False)
files.download('tsne_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Referensi
1.   https://taudata.blogspot.com/2022/05/nlptm-07.html
2.   https://github.com/plotly/dash-sample-apps/blob/main/apps/dash-nlp/ldacomplaints.py
3. https://github.com/taudataid/eLearning